<a href="https://colab.research.google.com/github/GirumSe/iCog-Internship-Tasks/blob/main/Semantic_Search/semantic_search_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install -U datasets -qq
!pip install -U sentence-transformers -qq
!pip install -U chromadb -qq
!pip install -U transformers -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requiremen

In [ ]:
import pandas as pd
import numpy as np

import torch

from datasets import load_dataset, get_dataset_config_names, concatenate_datasets, Features, Value
from sentence_transformers import SentenceTransformer,util
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import chromadb

from IPython.core.display import HTML, display
from IPython.display import clear_output
import string
from pprint import pprint
from IPython.utils import io


from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading the Dataset from HF and Preparing it

- i used only 5 sub-sets from the dataset.
- i merged the `title and content` of the news for more context, but their is a posibility to use `title` only, `content` only, `title, content and Description` and `title and Description` for experiment.

In [ ]:
configs = get_dataset_config_names("RealTimeData/bbc_news_alltime")
configs = configs[1:5]
all_datasets = []

for i in tqdm(configs):
  dataset = load_dataset("RealTimeData/bbc_news_alltime", i)
  dataset = dataset.map(lambda x: {'title_content': x['title'] + '\n' + x['content'], 'authors': [x["authors"]] if isinstance(x["authors"], str) else x["authors"]}, remove_columns=['authors', 'title', 'content'])
  dataset = dataset.cast(Features({'title_content': Value(dtype='string'), 'published_date': Value(dtype='string'), 'authors': [Value(dtype='string')], 'description': Value(dtype='string'), 'section': Value(dtype='string'), 'link': Value(dtype='string'), 'top_image': Value(dtype='string')}))
  all_datasets.append(dataset)

data = concatenate_datasets([d['train'] for d in all_datasets])

  0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1469 [00:00<?, ? examples/s]

Map:   0%|          | 0/1469 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1469 [00:00<?, ? examples/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.57s/it]

...

 75%|███████▌  | 3/4 [00:10<00:03,  3.39s/it]

Generating train split:   0%|          | 0/756 [00:00<?, ? examples/s]

Map:   0%|          | 0/756 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/756 [00:00<?, ? examples/s]

100%|██████████| 4/4 [00:15<00:00,  3.97s/it]


In [ ]:
data = concatenate_datasets([d['train'] for d in all_datasets])
df = pd.DataFrame(data)
df.fillna('', inplace=True) # Replace None with empty strings

# Prepare the metadata and documents
metadatas = df[['title_content', 'published_date', 'authors', 'description', 'section', 'link']].to_dict('records')

# Iterate over the metadatas and convert list values to strings
for metadata in metadatas:
    for key, value in metadata.items():
        if isinstance(value, list):
            metadata[key] = ', '.join(value) # Join list items with a comma and space

In [ ]:
documents = df['title_content'].tolist()
ids = [str(i) for i in range(len(df))]

# Vector DB preparation and Embeddings

- used persistant Chroma clint cause if i load a huge dataset i don't have to create it everytime since it isn't just in-memory and i can save it for other uses in sqlite format.
- I used Hierarchical Navigable Small Worlds(HNSW) native integration in chroma to impliment Approximate Nearest Neighbor (ANN).


In [ ]:
persist_directory = 'bbc_news_db'
client = chromadb.PersistentClient(path=persist_directory)
collection = client.get_or_create_collection("bbc_news", metadata={
    "hnsw:space": "cosine",
    "hnsw:construction_ef": 100,
    "hnsw:M": 16,
    "hnsw:search_ef": 10,
    "hnsw:num_threads": 4,
    "hnsw:resize_factor": 1.2,
    "hnsw:batch_size": 100,
    "hnsw:sync_threshold": 1000,})

- used the highest overall achiving sentence transformer `all-mpnet-base-v2` for sentence vectoriztion purpose.

In [ ]:
model_tokinizer = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
batch_size = 512
embeddings = []
for i in tqdm(range(0, len(df['title_content']), batch_size)):
    batch = df['title_content'][i:i+batch_size].tolist()
    batch_embeddings = model_tokinizer.encode(batch, device=device)
    embeddings.extend(batch_embeddings)

embeddings = [embedding.tolist() for embedding in embeddings]

100%|██████████| 8/8 [01:23<00:00, 10.43s/it]


In [ ]:
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
    embeddings=embeddings
)

# Query and Output

In [ ]:
query = input("Please enter your search query: ")
query_embedding = model_tokinizer.encode(query, device=device)

Please enter your search query: manchester united


- Used chromaDB's own query implimentation to do the query using Approximate Nearest Neighbor technique Hierarchical Navigable Small World (HNSW).

In [ ]:
results = collection.query(
    query_embeddings=query_embedding.tolist(),
    n_results=10,
    include=['embeddings', 'metadatas', 'documents', 'distances']
)

- The AutoTokenizer and AutoModelForSequenceClassification classes are used to load the tokenizer and model weights, respectively. The model is then moved to the appropriate device (CPU or GPU), as specified earlier in the code (device).

In [ ]:
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model = model.to(device)

- The tokenizer is used to convert the text pairs into input tensors suitable for the model, with padding and truncation applied to ensure uniform input sizes. The input tensors are moved to the same device as the model (device). The model is run in inference mode (torch.no_grad()), where it outputs logits representing the relevance scores for each text pair. The logits are then moved back to the CPU and converted to a NumPy array.
- The relevance scores are used to rank the documents. The results are sorted in descending order of relevance using np.argsort(scores)[::-1]. The final ranked_results list contains the search results ordered by their relevance to the user's query.

In [ ]:
texts = [[query, results['documents'][0][i]] for i, result in enumerate(results['metadatas'][0])]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
with torch.no_grad():
    scores = model(**inputs).logits.cpu().numpy()[:, 0]

ranked_results = [results['metadatas'][0][i] for i in np.argsort(scores)[::-1]]

- The title_content field in each result may contain newline characters `\n` that need to be rendered as line breaks in the output. The code replaces \n with the HTML line break tag `<br>` using the `replace()` method, ensuring the text is displayed with proper formatting in the browser.
- The formatted result is then displayed using the `HTML` function from the `IPython.display` module. styling and structure, ensures that the content is readable and well-presented.


In [ ]:
for result in ranked_results:
    formatted_content = result['title_content'].replace('\n', '<br>')
    display(HTML(f"""
        <div style="border:1px solid #ddd; padding:10px; margin:10px 0;">
            <p><strong>Description:</strong> {result['description']}</p>
            <p><strong>Published Date:</strong> {result['published_date']}</p>
            <a href="{result['link']}">Read more</a>
            <h3>{formatted_content}</h3>
        </div>
    """))

# References i used for this work
- Chroma-db Hierarchical Navigable Small Worlds(HSNW) [Documentation](https://cookbook.chromadb.dev/core/configuration/#examples)
- nice kaggle notebook with low level semantic search implimentation, and HTML [visualization](https://www.kaggle.com/code/maggieezzat/covid19-semantic-based-search-using-word-embedding#Ranking-documents).
- News Searching [idea](https://www.kaggle.com/code/shirshmall/project-ir-semantic-search-summarize-qna).